In [ ]:
#@markdown <center><h1>Install</h1></center>

%cd /content
!git clone -b totoro6 https://github.com/LucipherDev/ComfyUI /content/TotoroUI
!git clone -b totoro https://github.com/LucipherDev/ComfyUI-GGUF /content/TotoroUI/custom_nodes/TotoroUI-GGUF
!git clone -b totoro https://github.com/LucipherDev/ComfyUI-Inpaint-CropAndStitch /content/TotoroUI/custom_nodes/TotoroUI-Inpaint-CropAndStitch
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2
!pip install -q -r /content/TotoroUI/custom_nodes/TotoroUI-GGUF/requirements.txt
!apt -y install -qq aria2

import nodes

if not nodes.load_custom_node("custom_nodes/TotoroUI-GGUF"):
  raise Exception("Failed to load GGUF custom node")

if not nodes.load_custom_node("custom_nodes/TotoroUI-Inpaint-CropAndStitch"):
  raise Exception("Failed to load Inpaint-CropAndStitch custom node")

In [ ]:
#@markdown <center><h1>Load Models</h1></center>

import torch
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_differential_diffusion

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UnetLoaderGGUF = NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
DifferentialDiffusion = nodes_differential_diffusion.NODE_CLASS_MAPPINGS["DifferentialDiffusion"]()

print(f"Downloading Flux1-fill-dev-Q4_K_S...")
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/YarvixPA/FLUX.1-Fill-dev-gguf/resolve/main/flux1-fill-dev-Q4_K_S.gguf -d /content/TotoroUI/models/unet -o flux1-fill-dev-Q4_K_S.gguf

print("Downloading VAE...")
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft

print("Downloading Clips...")
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

with torch.inference_mode():
    print("Loading Clips...")
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    print("Loading VAE...")
    vae = VAELoader.load_vae("ae.sft")[0]
    print(f"Loading Flux1-fill-dev-Q4_K_S...")
    unet = UnetLoaderGGUF.load_unet(f"flux1-fill-dev-Q4_K_S.gguf")[0]
    unet = DifferentialDiffusion.apply(unet)[0]

    unet_f, clip_f = unet, clip

print("All Models Loaded!")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import re
import os
import random
import numpy as np
from google.colab import files, output
from IPython.display import HTML, display
from PIL import Image, ImageOps
import io
import base64

import nodes
from totoro_extras import nodes_flux
from totoro_extras import nodes_mask
from totoro import model_management

CLIPTextEncodeFlux = nodes_flux.NODE_CLASS_MAPPINGS["CLIPTextEncodeFlux"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]()
ImagePadForOutpaint = NODE_CLASS_MAPPINGS["ImagePadForOutpaint"]()
InpaintModelConditioning = NODE_CLASS_MAPPINGS["InpaintModelConditioning"]()
KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
ImageToMask = nodes_mask.NODE_CLASS_MAPPINGS["ImageToMask"]()
MaskToImage = nodes_mask.NODE_CLASS_MAPPINGS["MaskToImage"]()
GrowMask = nodes_mask.NODE_CLASS_MAPPINGS["GrowMask"]()
InpaintCrop = NODE_CLASS_MAPPINGS["InpaintCrop"]()
InpaintStitch = NODE_CLASS_MAPPINGS["InpaintStitch"]()
InpaintExtendOutpaint = NODE_CLASS_MAPPINGS["InpaintExtendOutpaint"]()
ImageCompositeMasked = nodes_mask.NODE_CLASS_MAPPINGS["ImageCompositeMasked"]()

In [ ]:
# @markdown <center><h1>Functions</h1></center>

interface = """
<style>#currentValues,.button{color:#fff;font-weight:700}body{background-color:#2d2d2d;font-family:Arial,sans-serif;color:#fff;margin:0;display:flex;flex-direction:column;align-items:center;justify-content:center}#buttonContainer,#toolbar{display:flex;flex-wrap:wrap;justify-content:center;align-items:center;margin:10px auto;background-color:#3c3f41;border-radius:8px;width:1024px;padding:10px 0;box-shadow:0 4px 10px rgba(0,0,0,.3)}.button{padding:10px 15px;margin:5px;font-size:14px;cursor:pointer;border-radius:5px;background-color:#5e5e5e;border:0;transition:.3s}.button:hover{background-color:#757575}.slider-container{display:flex;align-items:center;margin:10px 0;width:100%;padding:0 15px}.slider-container label{margin-right:10px}.slider-container input[type=range]{flex-grow:1;cursor:pointer}#currentValues{margin:15px 0;padding:10px;text-align:center}</style>
<div id="buttonContainer">
  <button class="button" id="resetButton">Reset</button>
  <button class="button" id="updateButton">Update</button>
</div>
<div id="toolbar">
  <div class="slider-container">
    <label for="topSlider">Top:</label>
    <input id="topSlider" type="range" min="0" max="1024" value="0">
  </div>
  <div class="slider-container">
    <label for="leftSlider">Left:</label>
    <input id="leftSlider" type="range" min="0" max="1024" value="0">
  </div>
  <div class="slider-container">
    <label for="bottomSlider">Bottom:</label>
    <input id="bottomSlider" type="range" min="0" max="1024" value="0">
  </div>
  <div class="slider-container">
    <label for="rightSlider">Right:</label>
    <input id="rightSlider" type="range" min="0" max="1024" value="0">
  </div>
  <div class="slider-container">
    <label for="featherSlider">Feather:</label>
    <input id="featherSlider" type="range" min="0" max="512" value="0">
  </div>
</div>
<div id="currentValues">Current Values: Top:0, Left:0, Bottom:0, Right:0, Feather:0</div>
<script>const sliders={top:document.getElementById("topSlider"),left:document.getElementById("leftSlider"),bottom:document.getElementById("bottomSlider"),right:document.getElementById("rightSlider"),feather:document.getElementById("featherSlider")},resetButton=document.getElementById("resetButton"),updateButton=document.getElementById("updateButton"),currentValues=document.getElementById("currentValues");let settings={top:0,left:0,bottom:0,right:0,feather:0};function updateSettings(){settings.top=parseInt(sliders.top.value),settings.left=parseInt(sliders.left.value),settings.bottom=parseInt(sliders.bottom.value),settings.right=parseInt(sliders.right.value),settings.feather=parseInt(sliders.feather.value),updateCurrentValues()}function updateCurrentValues(){currentValues.textContent=`Current Values: Top:${settings.top}, Left:${settings.left}, Bottom:${settings.bottom}, Right:${settings.right}, Feather:${settings.feather}`}function resetSettings(){Object.values(sliders).forEach((t=>t.value=0)),updateSettings()}function saveSettings(){google.colab.kernel.invokeFunction("notebook.update",[settings.top,settings.left,settings.bottom,settings.right,settings.feather],{})}Object.values(sliders).forEach((t=>t.addEventListener("input",updateSettings))),resetButton.addEventListener("click",resetSettings),updateButton.addEventListener("click",saveSettings),updateSettings();</script>
"""

top = 0
left = 0
bottom = 0
right = 0
feather = 0

# @markdown <ul><li><h2>Outpaint Mask</h2></ul>

input_image = "/content/test.png" # @param {"type":"string"}

# @markdown <ul><li><h2>Crop and Stitch</h2></li></li></ul>

context_expand_pixels = 20 # @param {"type":"slider","min":0,"max":256,"step":1}
context_expand_factor = 1 # @param {"type":"slider","min":1,"max":100,"step":0.01}
crop_mode = "ranged size" # @param ["ranged size", "forced size", "free size"] {"type":"string"}
rescale_algorithm = "bislerp" # @param ["nearest", "bilinear", "bicubic", "bislerp", "lanczos", "box", "hamming"]  {"type":"string"}
force_width =  1024 # @param {"type":"slider","min":512,"max":1024,"step":1}
force_height =  1024 # @param {"type":"slider","min":512,"max":1024,"step":1}
rescale_factor = 1 # @param {"type":"slider","min":0.01,"max":100,"step":0.01}
padding = 8 # @param ["8","16","32","64","128","256","512"] {"type":"raw"}
min_width = 512 # @param {"type":"slider","min":256,"max":512,"step":1}
min_height = 512 # @param {"type":"slider","min":256,"max":512,"step":1}
max_width = 768 # @param {"type":"slider","min":768,"max":1024,"step":1}
max_height = 768 # @param {"type":"slider","min":768,"max":1024,"step":1}

def update(top_value, left_value, bottom_value, right_value, feather_value):
    global top, left, bottom, right, feather
    top = top_value
    left = left_value
    bottom = bottom_value
    right = right_value
    feather = feather_value
    print(f"Values updated: Top:{top}, Left:{left}, Bottom:{bottom}, Right:{right}, Feather:{feather}")

def resize_image(img, limit, resize_by):
    width, height = img.size
    longest = max(width, height)
    shortest = min(width, height)

    if resize_by =="longest" and longest > limit:
        scale_factor = limit / float(longest)
        new_width = int(width * scale_factor)
        new_height = int(height * scale_factor)
        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

    elif resize_by == "shortest" and shortest > limit:
        scale_factor = limit / float(shortest)
        new_width = int(width * scale_factor)
        new_height = int(height * scale_factor)
        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

    return img

def save_image(decoded, path, name, download=False):
  full_path = os.path.abspath(os.path.join(path, name))
  Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save( full_path)

  img = Image.open(full_path)
  display(img)

  if download:
    files.download(full_path)

def img_tensor_to_np(img_tensor):
  img_tensor = img_tensor.clone() * 255.0
  return img_tensor.squeeze().numpy().astype(np.uint8)

def img_np_to_tensor(img_np_list):
  return torch.from_numpy(img_np_list.astype(np.float32) / 255.0).unsqueeze(0)

@torch.inference_mode()
def generate(pos_prompt, neg_prompt, fixed_seed, guidance, steps, cfg, sampler_name, scheduler, resize_before, size_limit, limit_by, use_crop_stitch, mask_expand, mask_blur, mask_blend, batch_size, auto_download):
  global unet, clip, unet_f, clip_f

  print("Prompt Received")

  pos_cond = CLIPTextEncodeFlux.encode(clip_f, pos_prompt, pos_prompt, guidance)[0]
  neg_cond = CLIPTextEncodeFlux.encode(clip_f, neg_prompt, neg_prompt, guidance)[0]

  image, mask = LoadImage.load_image(input_image)

  if resize_before:
    image_np = img_tensor_to_np(image)
    img = Image.fromarray(image_np)

    mask_image = MaskToImage.mask_to_image(mask)[0]
    mask_np = img_tensor_to_np(mask_image)
    mask_img = Image.fromarray(mask_np)

    img = resize_image(img, size_limit, limit_by)
    mask_img = resize_image(mask_img, size_limit, limit_by)

    image = img_np_to_tensor(np.array(img))
    mask = ImageToMask.image_to_mask(img_np_to_tensor(np.array(mask_img)), "red")[0]

  if use_crop_stitch:
    image, mask, _ = InpaintExtendOutpaint.inpaint_extend(image, mask, "pixels", top, 0, bottom, 0, left, 0, left, 0, mask)

    mask = GrowMask.expand_mask(mask, mask_expand, True)[0]

    stitch, image, mask = InpaintCrop.inpaint_crop_single_image(image, mask, context_expand_pixels, context_expand_factor, False, mask_blur, False, mask_blend, crop_mode, rescale_algorithm, force_width, force_height, rescale_factor, padding, min_width, min_height, max_width, max_height)

  else:
    image, mask = ImagePadForOutpaint.expand_image(image, left, top, right, bottom, feather)

  display(HTML(f"""
    <div style="display: flex; gap: 10px;">
        {"".join(f'<img src="data:image/png;base64,{base64.b64encode(io.BytesIO(Image.fromarray(img).save((buf:=io.BytesIO()), format="PNG") or buf.getvalue()).getvalue()).decode("utf-8")}" style="width: 512px;">' for img in [img_tensor_to_np(image), img_tensor_to_np(MaskToImage.mask_to_image(mask)[0])])}
    </div>
  """))

  pos_cond, neg_cond, latent_image = InpaintModelConditioning.encode(pos_cond, neg_cond, image, vae, mask, True)

  for i in range(0, batch_size):
    if fixed_seed == 0:
      seed = random.randint(0, 18446744073709551615)
    else:
      seed = fixed_seed

    print("Seed:", seed)

    sample = KSampler.sample(unet_f, seed, steps, cfg, sampler_name, scheduler, pos_cond, neg_cond, latent_image)[0]
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    decoded = ImageCompositeMasked.composite(image, decoded, 0, 0, True, mask)[0]

    if use_crop_stitch:
      decoded = InpaintStitch.inpaint_stitch_single_image(stitch, decoded, rescale_algorithm)[0]

    save_image(decoded, "/content", f"flux_fill_{seed}_{i}.png", auto_download)


output.register_callback('notebook.update', update)
display(HTML(interface))

In [ ]:
#@markdown <center><h1>Outpaint</h1></center>

positive_prompt = "" # @param {"type":"string"}
negative_prompt = "watermark, text, ugly, deformed, low quality" # @param {"type":"string"}
fixed_seed = 0 # @param {"type":"slider","min":0,"max":18446744073709552000,"step":1}
guidance = 30 # @param {"type":"slider","min":0,"max":50,"step":0.5}
steps = 20 # @param {"type":"slider","min":4,"max":50,"step":1}
cfg = 1 # @param {"type":"slider","min":0,"max":100,"step":0.1}
resize_before = True # @param {"type":"boolean"}
size_limit = 1024 # @param {"type":"slider","min":256,"max":2048,"step":1}
limit_by = "shortest" # @param ["longest","shortest"]
use_crop_stitch = True # @param {"type":"boolean"}
mask_expand = 0 # @param {"type":"slider","min":-256,"max":256,"step":1}
mask_blur = 16 # @param {"type":"slider","min":0,"max":64,"step":1}
mask_blend = 16 # @param {"type":"slider","min":0,"max":32,"step":1}
batch_size = 1 # @param {"type":"slider","min":1,"max":20,"step":1}
sampler_name = "euler" # @param ["euler","heun","heunpp2","heunpp2","dpm_2","lms","dpmpp_2m","ipndm","deis","ddim","uni_pc","uni_pc_bh2"]
scheduler = "simple" # @param ["normal","sgm_uniform","simple","ddim_uniform"]
auto_download = False # @param {"type":"boolean"}

generate(positive_prompt, negative_prompt, fixed_seed, guidance, steps, cfg, sampler_name, scheduler, resize_before, size_limit, limit_by, use_crop_stitch, mask_expand, mask_blur, mask_blend, batch_size, auto_download)